In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
path = '/kaggle/input/valid-dataset-adobe/valid_paths_data.csv'

In [3]:
import pandas as pd
processed_df = pd.read_csv(path)
processed_df

,id,date,likes,username,media,inferred company,hour,month,day of week,Year,content_processed,Link,Media Type,image_path
0,1,2020-12-12 00:47:00,1,TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,0,12,12,2020,spend weekend morning ham egg cheese wrap pair...,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...
1,2,2018-06-30 10:04:20,2750,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,10,6,30,2018,watch rapper freestyle hour,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...
2,3,2020-09-29 19:47:28,57,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,19,9,29,2020,canadian armenian community demands ban milita...,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...
3,4,2020-10-01 11:40:09,152,MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,11,10,1,2020,1st europe devastated covid19 italy redoubled ...,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...
4,5,2018-10-19 14:30:46,41,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,14,10,19,2018,congratulations pauletha butts presented beyon...,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,Photo,/kaggle/input/behavior-challenge-images/media1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295497,299996,2019-09-07 16:18:10,0,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,16,9,7,2019,barcelona star expected return valencia,https://pbs.twimg.com/media/ED397wUUYAEOV4K?fo...,Photo,/kaggle/input/behavior-challenge-images/media2...
295498,299997,2018-02-23 11:24:36,46,CBCOlympics,[Video(thumbnailUrl='https://pbs.twimg.com/med...,cbc,11,2,23,2018,kjeld nuis ned golden time mens 1000m beating ...,https://pbs.twimg.com/media/DWt21i8VMAE69_F.jpg,Video,/kaggle/input/behavior-challenge-images/media2...
295499,299998,2020-11-11 20:18:15,261,TeamCoachBuzz,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,20,11,11,2020,grateful officially join,https://pbs.twimg.com/media/EmkaGpzXIAEyHg8?fo...,Photo,/kaggle/input/behavior-challenge-images/media2...
295500,299999,2019-10-29 10:44:00,119,Independent,[Video(thumbnailUrl='https://pbs.twimg.com/med...,independent,10,10,29,2019,us air forces mysterious space plane landed re...,https://pbs.twimg.com/media/EICgLEQWwAIt3Qr.jpg,Video,/kaggle/input/behavior-challenge-images/media2...


In [4]:
from PIL import Image
Image.open(processed_df['image_path'][0]).convert('RGB').size

(680, 680)

In [5]:
!pip install transformers

In [6]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from tqdm import tqdm

def load_clip_model():
    # Determine the device (cuda if available, otherwise cpu)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load CLIP processor, tokenizer, and model
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").to(device)

    # Return the loaded components
    return processor, model, device, tokenizer

def process_batches(processor, tokenizer, model, device, image_paths, text_descriptions, batch_size):
    # Lists to store image and text embeddings
    image_embeddings = []
    text_embeddings = []

    # Use tqdm for the loop to display a progress bar
    for i in tqdm(range(0, len(image_paths), batch_size)):
        # Load a batch of images and corresponding text descriptions
        batch_images = [Image.open(image_path).convert('RGB') for image_path in image_paths[i:i + batch_size]]
        text_description = text_descriptions[i:i + batch_size]
        text_description = [str(i) for i in text_description]

        # Tokenize text descriptions and move to the specified device
        text_inputs = tokenizer(text_description, return_tensors="pt", padding=True, truncation=True)
        text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

        # Encode image using the CLIP processor
        image_inputs = processor(images=batch_images, return_tensors="pt")
        image_inputs = {key: value.to(device) for key, value in image_inputs.items()}

        # Forward pass through the model to obtain embeddings
        with torch.no_grad():
            text_embedding = model.get_text_features(**text_inputs)
            image_embedding = model.get_image_features(**image_inputs)

        # Append the embeddings to the respective lists
        image_embeddings.append(image_embedding)
        text_embeddings.append(text_embedding)

    # Concatenate the embeddings along the specified dimension
    image_embeddings = torch.cat(image_embeddings, dim=0)
    text_embeddings = torch.cat(text_embeddings, dim=0)

    # Return the concatenated image and text embeddings
    return image_embeddings, text_embeddings


In [7]:
def get_embeddings(df, batch_size):
    # Extract image paths and create text descriptions from the DataFrame
    image_paths = df['image_path'].values.tolist()
    text_descriptions_df = df['username'] + ' ' + df['inferred company'] + ' ' + df['content_processed']
    text_descriptions = text_descriptions_df.values.tolist()

    # Load CLIP model components
    processor, model, device, tokeniser = load_clip_model()

    # Process batches to obtain image and text embeddings
    image_embeddings, text_embeddings = process_batches(processor, tokeniser, model, device, image_paths, text_descriptions, batch_size)

    # Print the shapes of the obtained embeddings
    print("Image Embeddings Shape:", image_embeddings.shape)
    print("Text Embeddings Shape:", text_embeddings)

    # Return the image and text embeddings
    return image_embeddings, text_embeddings


In [8]:
img_emb, text_emb = get_embeddings(processed_df, batch_size = 32)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

  0%|          | 24/9235 [00:28<2:27:40,  1.04it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 9235/9235 [2:27:54<00:00,  1.04it/s]

Image Embeddings Shape: torch.Size([295502, 512])
Text Embeddings Shape: tensor([[ 0.0172,  0.1405, -0.2464,  ..., -0.2386, -0.1563,  0.2386],
        [ 0.1155,  0.1130, -0.4140,  ..., -0.1418,  0.0552, -0.2318],
        [ 0.1452, -0.0988,  0.0885,  ..., -0.3298, -0.2161, -0.1648],
        ...,
        [-0.1245, -0.5266,  0.1277,  ...,  0.6149,  0.4896,  0.3379],
        [-0.1319,  0.3462, -0.0075,  ...,  0.2928, -0.3788, -0.2042],
        [-0.2055,  0.3249, -0.1462,  ..., -0.1104, -0.1558,  0.3209]],
       device='cuda:0')


In [9]:
print(img_emb)

tensor([[-0.2469, -0.1999,  0.1567,  ..., -0.2530,  0.1175,  0.6541],
        [-0.3988, -0.2993, -0.2622,  ..., -0.0551,  0.0517, -0.0600],
        [-0.0914, -0.1765, -0.0289,  ...,  0.0418,  0.3370, -0.3859],
        ...,
        [-0.3025, -0.4208,  0.2079,  ...,  0.3318,  0.7197,  0.6046],
        [-0.1804, -0.3546, -0.2603,  ...,  0.5347, -0.0272,  0.3734],
        [ 0.1689,  0.2614, -0.1780,  ...,  0.1915,  0.1814,  0.7319]],
       device='cuda:0')


In [10]:
result_tensor_embed = torch.cat((img_emb, text_emb), dim=1)
# result_tensor_embed = img_emb

In [11]:
import numpy as np
result_numpy_embed = result_tensor_embed.cpu().numpy()
np.save('/kaggle/working/CLIP_Embeds.npy', result_numpy_embed)